In [19]:
import numpy as np
import random
import node


In [40]:
operators=[np.add, np.subtract, np.multiply, np.sin, np.cos, np.exp]
one_arg_op=[np.sin, np.cos, np.exp]

def is_operator(val):
        return val in operators

In [ ]:
class Node:

    op_list = {
        np.add: '+',
        np.subtract: '-',
        np.multiply: '*',
        np.sin: 'sin',
        np.cos: 'cos',
        np.exp: 'exp',  
    }
    def __init__(self, value=None, feature_index=None, left=None, right=None):
        self.value = value 
        self.feature_index = feature_index
        self.left = left
        self.right = right
        self.complexity = self.calculate_complexity()
    
    def calculate_complexity(self):
        if not is_operator(self.value):
            return 0
        
        if self.value in one_arg_op:
            if self.value == np.exp:
                return 3 + (self.left.calculate_complexity() if self.left else 0)
            else:
                return 2 + (self.left.calculate_complexity() if self.left else 0)

        left_complexity = self.left.calculate_complexity() if self.left else 0
        right_complexity = self.right.calculate_complexity() if self.right else 0
        return 1 + left_complexity + right_complexity

    def evaluate(self,x=None):
        """
        calculates the result of the equation with the input x
    
        Args:
            x : the input of the equation

        Returns:
            value : the result of the calculation.
        """
        # first check if it is an operator 
        # if not proceed
        
        if not is_operator(self.value):
            #print("index",self.feature_index)
            if self.feature_index!= None:
                #print("ben inputun indexiyim")
                return x[self.feature_index]
            else:
                #print("ben bir sayıyım")
                return self.value
        # if it is an operator
        if self.value in one_arg_op:
            operand_value = self.left.evaluate(x)
            #print("tek",operand_value)
            return self.value(operand_value)
        
        left_value = self.left.evaluate(x)
        right_value = self.right.evaluate(x)
        #print(left_value)
        #print(right_value)
        return self.value(left_value, right_value)
    def __str__(self):
        if not is_operator(self.value):
            if self.feature_index != None:
                return f"x[{self.feature_index}]"
            return str(self.value)

        operator_symbol = self.op_list[self.value]

        if self.value in {np.sin, np.cos, np.log, np.exp}:
            return f"{operator_symbol}({self.left})"

        return f"({self.left} {operator_symbol} {self.right})"

In [42]:
meric = Node(value=np.add,left=Node(feature_index=0),right=Node(value=np.multiply,left=Node(value=2),right=Node(value=np.sin,left=Node(value=np.add,left=Node(feature_index=1),right=Node(value=10)))))
print(meric)

(x[0] + (2 * sin((x[1] + 10))))


In [43]:
x=[2,3]
print(meric.evaluate(x))

2.840334073653282


In [ ]:
def random_tree(depth, num_features):
    """
    

    Args:
        x : the input of the equation

    Returns:
        value : the result of the calculation.
    """
    if depth == 0:
        if random.random() < 0.5:
            return Node(feature_index=random.randint(0, num_features - 1))
        else:
            return Node(value=np.random.normal(0,1))

    operator = random.choice(operators)
    node = Node(value=operator)

    if operator in one_arg_op:
        node.left = random_tree(depth - 1, num_features)
        node.right = None
    else:
        node.left = random_tree(depth - 1, num_features)
        node.right = random_tree(depth - 1, num_features)

    return node

In [45]:
bayram=random_tree(5,2)
print(bayram)

(((((x[0] - x[0]) - exp(6)) - ((5 * 6) * cos(4))) + exp(((x[0] + x[1]) - (x[0] - x[0])))) + (((exp(8) * (8 - x[1])) - ((x[1] - 9) + cos(3))) - sin(exp((x[1] - x[0])))))


In [46]:
bayram.evaluate(x)

np.float64(14675.962820650491)

In [78]:
def create_population(num_peop,depth,num_features):
    population = []
    num_ones = num_peop//2
    for i in range(num_ones):
        baby=random_tree(1,num_features)
        population.append(baby)
    for i in range(num_peop-num_ones):
        baby=random_tree(depth,num_features)
        population.append(baby)
    return population

In [86]:
new_population=create_population(4,3,2)
for i in range(len(new_population)):
    print(new_population[i])

cos(x[0])
(x[1] - x[0])
(((x[0] + x[0]) + (x[0] + x[0])) - cos((x[0] - x[0])))
(cos(sin(x[0])) - (cos(-0.39726116844588705) * (x[0] + x[1])))


In [90]:
old_population = create_population(4,3,2)
for i in range(len(old_population)):
    print(old_population[i])

(1.0602829687704254 + x[1])
(x[0] * 0.15453349013771153)
(((x[0] + -0.6648895361303597) + cos(x[1])) + ((x[1] - x[0]) - sin(-0.791398794248259)))
(sin(cos(x[1])) + ((1.3540560734533116 * -0.978236027265072) * sin(x[1])))


In [52]:
data = np.load('../data/problem_0.npz')
x = data['x']
y = data['y']
num_features=len(x)
depth=3

In [ ]:
def cost(genome,x,y):
    predictions = np.array([genome.evaluate(x[:, i]) for i in range(x.shape[1])])
    mse = np.mean((predictions - y) ** 2)
    return mse

In [80]:
c=cost(new_population[0],x,y)
print(c)

3.3988605675121466


In [69]:
def cost_population(population, x, y):
    costs = np.array([cost(population[j],x,y) for j in range(len(population))])
    return costs

In [87]:
cost_pop = cost_population(new_population,x,y)
print(cost_pop)

[ 3.86949114 13.77960458 31.05455319 13.38730661]


In [91]:
cost_pop2 = cost_population(old_population,x,y)
print(cost_pop2)

[4.57210022 2.4326046  4.2932951  4.46855433]


In [95]:
def migration(population_1,population_2,num_peop,x,y):
    costs_1 = cost_population(population_1,x,y)
    costs_2 = cost_population(population_2,x,y)
    best_1, worst_1 = utils.tournament_selection(costs_1,num_peop)
    best_2, worst_2 = utils.tournament_selection(costs_2,num_peop)
    elements_1 = [population_1[i] for i in best_1]
    elements_2 = [population_2[i] for i in best_2]
    for i, idx in enumerate(best_1):
        population_1[idx] = elements_2[i]
    for i, idx in enumerate(best_2):
        population_2[idx] = elements_1[i]
    return population_1,population_2

In [97]:
pop1,pop2=migration(new_population,old_population,2,x,y)
for i in range(len(pop1)):
    print(pop1[i])

(x[0] * 0.15453349013771153)
(x[1] - x[0])
(((x[0] + x[0]) + (x[0] + x[0])) - cos((x[0] - x[0])))
(((x[0] + -0.6648895361303597) + cos(x[1])) + ((x[1] - x[0]) - sin(-0.791398794248259)))


In [98]:
for i in range(len(pop2)):
    print(pop2[i])

(1.0602829687704254 + x[1])
cos(x[0])
(cos(sin(x[0])) - (cos(-0.39726116844588705) * (x[0] + x[1])))
(sin(cos(x[1])) + ((1.3540560734533116 * -0.978236027265072) * sin(x[1])))


In [23]:
x,y=utils.tournament_selection([10,5,8,9],2)
print(x)
print(y)

[1 2]
[0 3]


In [66]:
import numpy as np
from node import *

operators=[np.add, np.subtract, np.multiply, np.sin, np.cos, np.exp]
one_arg_op=[np.sin, np.cos, np.exp]

In [67]:
problem_0 = np.load("../data/problem_2.npz")
x = problem_0["x"]
y = problem_0["y"]
print("x.shape:", x.shape)
print("y.shape:", y.shape)

x.shape: (3, 5000)
y.shape: (5000,)


In [50]:
my_population = create_population(120,5,x.shape[0])

In [51]:
simplified = simplify(my_population, x, y)
print("simplified.shape:", len(simplified))

simplified.shape: 114


In [52]:
cost_pop = cost_population(simplified[0:100],x,y)
print("Cost population: ", cost_pop.mean())
print("Best cost: ", cost_pop.min())

Cost population:  9.632750342517252e+116
Best cost:  29616601983729.535


In [53]:
NUM_GENERATIONS = 10000
operators=[np.add, np.subtract, np.multiply, np.sin, np.cos, np.exp]
one_arg_op=[np.sin, np.cos, np.exp]

for _ in range(NUM_GENERATIONS):
    best_ind, worst_ind = tournament_selection(cost_pop,10)
    # Mutate the best individual and replace worst individuals
    for j in range(0,len(worst_ind)//2,2):
        parent1 = simplified[best_ind[np.random.choice(len(worst_ind))]]
        parent2 = simplified[best_ind[np.random.choice(len(worst_ind))]]
        child1, child2 = crossover(parent1,parent2)
        
        simplified[worst_ind[j]] = child1
        simplified[worst_ind[j+1]] = child2

In [54]:
cost_pop_1 = cost_population(simplified,x,y)
print("Cost population: ", cost_pop_1.mean())
print("Best cost: ", cost_pop_1.min())

Cost population:  2.4298258469666409e+263
Best cost:  29595267646038.98


In [55]:
my_population_2 = create_population(120,3,x.shape[0])
simplified_2 = simplify(my_population_2, x, y)
print("simplified.shape:", len(simplified_2))

simplified.shape: 119


In [56]:
cost_pop_2 = cost_population(simplified_2[0:100],x,y)
print("Cost population: ", cost_pop_2.mean())
print("Best cost: ", cost_pop_2.min())

Cost population:  2.3829934962022125e+123
Best cost:  29616658628611.88


In [57]:
NUM_GENERATIONS = 10000
operators=[np.add, np.subtract, np.multiply, np.sin, np.cos, np.exp]
one_arg_op=[np.sin, np.cos, np.exp]

for _ in range(NUM_GENERATIONS):
    best_ind_2, worst_ind_2 = tournament_selection(cost_pop_2,10)
    # Mutate the best individual and replace worst individuals
    for j in range(0,len(worst_ind_2)//2,2):
        parent1_2 = simplified_2[best_ind_2[np.random.choice(len(worst_ind_2))]]
        parent2_2 = simplified_2[best_ind_2[np.random.choice(len(worst_ind_2))]]
        child1_2, child2_2 = crossover(parent1_2,parent2_2)
        
        simplified_2[worst_ind_2[j]] = child1_2
        simplified_2[worst_ind_2[j+1]] = child2_2

In [58]:
cost_pop_3 = cost_population(simplified_2,x,y)
print("Cost population: ", cost_pop_3.mean())
print("Best cost: ", cost_pop_3.min())

Cost population:  4.275927397203199e+124
Best cost:  29616096511237.004


In [60]:
pop1,pop2=migration(simplified,simplified_2,10,x,y)
cost_pop_4 = cost_population(pop1,x,y)
print("Cost population: ", cost_pop_4.mean())
print("Best cost: ", cost_pop_4.min())

Cost population:  2.4298258469666409e+263
Best cost:  29595267646038.98


In [61]:
cost_pop_5 = cost_population(pop2,x,y)
print("Cost population: ", cost_pop_5.mean())
print("Best cost: ", cost_pop_5.min())

Cost population:  4.275927397203199e+124
Best cost:  29616096511237.004


In [62]:
NUM_GENERATIONS = 10000
operators=[np.add, np.subtract, np.multiply, np.sin, np.cos, np.exp]
one_arg_op=[np.sin, np.cos, np.exp]

for _ in range(NUM_GENERATIONS):
    best_ind_3, worst_ind_3 = tournament_selection(cost_pop_4,10)
    # Mutate the best individual and replace worst individuals
    for j in range(0,len(worst_ind_3)//2,2):
        parent1_3 = pop1[best_ind_3[np.random.choice(len(worst_ind_3))]]
        parent2_3 = pop1[best_ind_3[np.random.choice(len(worst_ind_3))]]
        child1_3, child2_3 = crossover(parent1_3,parent2_3)
        
        pop1[worst_ind_3[j]] = child1_3
        pop1[worst_ind_3[j+1]] = child2_3

In [63]:
cost_pop_6 = cost_population(pop1,x,y)
print("Cost population: ", cost_pop_6.mean())
print("Best cost: ", cost_pop_6.min())

Cost population:  29616768786546.57
Best cost:  29595267646038.98


In [64]:
NUM_GENERATIONS = 10000
operators=[np.add, np.subtract, np.multiply, np.sin, np.cos, np.exp]
one_arg_op=[np.sin, np.cos, np.exp]

for _ in range(NUM_GENERATIONS):
    best_ind_4, worst_ind_4 = tournament_selection(cost_pop_5,10)
    # Mutate the best individual and replace worst individuals
    for j in range(0,len(worst_ind_4)//2,2):
        parent1_4 = pop2[best_ind_4[np.random.choice(len(worst_ind_4))]]
        parent2_4 = pop2[best_ind_4[np.random.choice(len(worst_ind_4))]]
        child1_4, child2_4 = crossover(parent1_4,parent2_4)
        
        pop1[worst_ind_4[j]] = child1_4
        pop1[worst_ind_4[j+1]] = child2_4

In [65]:
cost_pop_7 = cost_population(pop2,x,y)
print("Cost population: ", cost_pop_7.mean())
print("Best cost: ", cost_pop_7.min())

Cost population:  4.275927397203199e+124
Best cost:  29616096511237.004


In [ ]:
def mutation_w_sa(individual, feature_count,alpha):
    child = mutation(individual, feature_count)
    child.fitness = cost(child.genome)
    if child.fitness > individual.fitness:
        individual.T *=alpha
        return child
    else: 
        p= np.exp((child.fitness-individual.fitness)/(alpha*individual.T))
        if np.random.random() < p:
            return None
        else:
            individual.T *=alpha
            return child

In [ ]:
def age_update(population):
    for i in population:
        i.age += 1

In [1]:
import numpy as np
from src.Individual import Individual
from src.meric_node import *

operators=[np.add, np.subtract, np.multiply, np.sin, np.cos, np.exp, np.abs]
one_arg_op=[np.sin, np.cos, np.exp, np.abs]

In [2]:
problem_0 = np.load("data/problem_0.npz")
x = problem_0["x"]
y = problem_0["y"]
print("x.shape:", x.shape)
print("y.shape:", y.shape)

x.shape: (2, 1000)
y.shape: (1000,)


In [3]:
NUM_POPULATION = 20
my_population = create_population(NUM_POPULATION,5,x.shape[0])

In [4]:
assign_population_fitness(my_population,x,y)
print(len(my_population))
# print("Cost population: ", cost_pop.mean())
# print("Best cost: ", cost_pop.min())

20


In [5]:
for i in range(len(my_population)):
    print(my_population[i])
    print(my_population[i].genome.complexity)

Individual(genome=(x[1] * x[1]), fitness=np.float64(3.5312243425034984), age=0, T=1)
3
Individual(genome=exp(x[1]), fitness=np.float64(4.8881455049679285), age=0, T=1)
6
Individual(genome=abs(x[1]), fitness=np.float64(3.620156753500731), age=0, T=1)
2
Individual(genome=(x[1] - [-0.05266735]), fitness=np.float64(3.7199254545083495), age=0, T=1)
3
Individual(genome=([-0.94022194] - [-0.74581029]), fitness=np.float64(3.4654429798353505), age=0, T=1)
3
Individual(genome=(x[1] * x[0]), fitness=np.float64(4.586575180257536), age=0, T=1)
3
Individual(genome=cos([2.40156143]), fitness=np.float64(4.053507156875347), age=0, T=1)
4
Individual(genome=cos([1.00685666]), fitness=np.float64(3.6054258322227004), age=0, T=1)
4
Individual(genome=cos([2.61209496]), fitness=np.float64(4.271528042118085), age=0, T=1)
4
Individual(genome=(x[0] + [-0.13884782]), fitness=np.float64(6.7950422913120905), age=0, T=1)
3
Individual(genome=exp(sin((sin(abs([-2.29858336])) - (exp(x[0]) - sin([-1.24730085]))))), fitn

In [6]:
my_population[0].genome.left=Node(value=np.add,left=Node(feature_index=1),right=Node(value=10))

In [7]:
my_population[0].genome.left

(x[1] + 10)

In [8]:
for i in range(len(my_population)):
    print(my_population[i])
    print(my_population[i].genome.complexity)

Individual(genome=((x[1] + 10) * x[1]), fitness=np.float64(3.5312243425034984), age=0, T=1)
5
Individual(genome=exp(x[1]), fitness=np.float64(4.8881455049679285), age=0, T=1)
6
Individual(genome=abs(x[1]), fitness=np.float64(3.620156753500731), age=0, T=1)
2
Individual(genome=(x[1] - [-0.05266735]), fitness=np.float64(3.7199254545083495), age=0, T=1)
3
Individual(genome=([-0.94022194] - [-0.74581029]), fitness=np.float64(3.4654429798353505), age=0, T=1)
3
Individual(genome=(x[1] * x[0]), fitness=np.float64(4.586575180257536), age=0, T=1)
3
Individual(genome=cos([2.40156143]), fitness=np.float64(4.053507156875347), age=0, T=1)
4
Individual(genome=cos([1.00685666]), fitness=np.float64(3.6054258322227004), age=0, T=1)
4
Individual(genome=cos([2.61209496]), fitness=np.float64(4.271528042118085), age=0, T=1)
4
Individual(genome=(x[0] + [-0.13884782]), fitness=np.float64(6.7950422913120905), age=0, T=1)
3
Individual(genome=exp(sin((sin(abs([-2.29858336])) - (exp(x[0]) - sin([-1.24730085]))))

In [9]:
# Sort the population by fitness (ascending), then take the top 5
top_5_individuals = top_n_individuals(my_population, 3)

# Print the top 5 individuals with the minimum fitness
for i, individual in enumerate(top_5_individuals, 1):
    print(f"Top {i}: Fitness = {individual.fitness}: Genome = {individual.genome}")

Top 1: Fitness = 3.4654429798353505: Genome = ([-0.94022194] - [-0.74581029])
Top 2: Fitness = 3.5312243425034984: Genome = ((x[1] + 10) * x[1])
Top 3: Fitness = 3.5671104218649: Genome = (cos(((cos(x[1]) + abs([-0.01683803])) - abs((x[0] + [0.69740563])))) * cos((sin(sin(x[0])) + sin(sin(x[0])))))


In [10]:
NUM_GENERATIONS = 30
DEDUPE_INTERVAL = 5
operators=[np.add, np.subtract, np.multiply, np.sin, np.cos, np.exp]
one_arg_op=[np.sin, np.cos, np.exp]
XOVER = 0.5
MUTATION = 0.5

for generation in range(NUM_GENERATIONS):
    age_population(my_population)
    assign_population_fitness(my_population,x,y)

    # Kill eldest individuals
    kill_eldest(my_population, 10)


    best_ind = tournament_selection(my_population,3,10)

    
    for _ in range(10): # CHECK
        if random.random() < XOVER:
            # Xover the best individual and replace worst individuals
            parent1 = my_population[best_ind[np.random.choice(len(best_ind))]]
            parent2 = my_population[best_ind[np.random.choice(len(best_ind))]]
            child1, child2 = crossover(parent1,parent2)
            my_population.append(child1)
            my_population.append(child2)

        if random.random() < MUTATION:
            # Mutate best individuals and replace worst individuals
            parent = my_population[best_ind[np.random.choice(len(best_ind))]]
            try:
                child = mutation_w_sa(parent, x.shape[0],0.95,x,y)
                my_population.append(child)
            except:
                pass
    assign_population_fitness(my_population,x,y)
        
    # # Deduplicate every few iterations
    # if generation % DEDUPE_INTERVAL == 0:
    #     print("Deduplicating population")
    #     my_population = list(dict.fromkeys(my_population))
    #     print("Population size: ", len(my_population))

In [11]:
for i in range(len(my_population)):
    print(my_population[i])

Individual(genome=[-0.94022194], fitness=np.float64(4.422096467281613), age=10, T=1)
Individual(genome=x[0], fitness=np.float64(0.01067566400550203), age=10, T=1)
Individual(genome=[-0.94022194], fitness=np.float64(4.422096467281613), age=10, T=1)
Individual(genome=x[0], fitness=np.float64(0.01067566400550203), age=10, T=1)
Individual(genome=x[0], fitness=np.float64(0.01067566400550203), age=10, T=1)
Individual(genome=x[0], fitness=np.float64(0.01067566400550203), age=10, T=1)
Individual(genome=([-0.94022194] - [-0.74581029]), fitness=np.float64(3.4654429798353505), age=10, T=1)
Individual(genome=([-0.94022194] - [-0.74581029]), fitness=np.float64(3.4654429798353505), age=10, T=1)
Individual(genome=x[0], fitness=np.float64(0.01067566400550203), age=10, T=1)
Individual(genome=x[0], fitness=np.float64(0.01067566400550203), age=10, T=1)
Individual(genome=x[0], fitness=np.float64(0.01067566400550203), age=10, T=1)
Individual(genome=[-0.94022194], fitness=np.float64(4.422096467281613), age=

In [12]:
top_5_individuals = top_n_individuals(my_population, 5)

# Print the top 5 individuals with the minimum fitness
for i, individual in enumerate(top_5_individuals, 1):
    print(f"Top {i}: Fitness = {individual.fitness}: Genome = {individual.genome}")

Top 1: Fitness = 0.01067566400550203: Genome = x[0]
Top 2: Fitness = 0.01067566400550203: Genome = x[0]
Top 3: Fitness = 0.01067566400550203: Genome = x[0]
Top 4: Fitness = 0.01067566400550203: Genome = x[0]
Top 5: Fitness = 0.01067566400550203: Genome = x[0]
